In [51]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup


def get_url(campo, localizacao):
    template = 'https://br.indeed.com/jobs?q={}&l={}'
    campo = campo.replace(' ', '+')
    localizacao = localizacao.replace(' ', '+')
    url = template.format(campo, localizacao)
    return url

def get_record(card):
    job_title = card.span.get('title')
    company = card.find('span', 'companyName').text.strip()
    job_location = card.find('div', 'companyLocation').text.strip()
    salary_tag = card.find('span', 'salaryText')
    if salary_tag:
        salary = salary_tag.text.strip()
    else:
        salary = ''  
    
        
    record = (job_title, company, job_location,salary_tag)
    return record





def main(position, location):
    records = []
    url = get_url(position, location)
    
    # extract the job data
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'job_seen_beacon')
        for card in cards:
            record = get_record(card)
            records.append(record)
        try:
            url = 'https://www.br.indeed.com' + soup.find('a', {'aria-label': 'Próximo'}).get('href')
        except AttributeError:
            break
        
    # save the job data
    with open('resultsNovo5.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Área', 'Empresa', 'Localização','Salário'])
        writer.writerows(records)


In [55]:
main('Estagio','Brasil')